In [5]:
%reset -f

In [3]:
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import ee
ee.Initialize(project='ee-andrewfullhart')

ic = ee.ImageCollection('NASA/NEX-DCP30')

out_file = '/content/drive/MyDrive/Colab Notebooks/NEX_Ensemble_Stats_Precip_MEAN.csv'

study_area = ee.FeatureCollection('users/andrewfullhart/SW_Study_Area')

model_list = ee.List(['ACCESS1-0', 'bcc-csm1-1', 'BNU-ESM', 'CanESM2', 'CCSM4', 'CESM1-BGC', 'CNRM-CM5', 'CSIRO-Mk3-6-0', 'GFDL-CM3', 'GFDL-ESM2G', 'GFDL-ESM2M', 'inmcm4', 'IPSL-CM5A-LR', 'IPSL-CM5A-MR', 'MIROC-ESM', 'MIROC-ESM-CHEM', 'MIROC5', 'MPI-ESM-LR', 'MPI-ESM-MR', 'MRI-CGCM3', 'NorESM1-M'])
model_order = ee.List.sequence(0, ee.Number(model_list.size()).subtract(1))

ndays_months = ee.List([31, 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31])
order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])

reducer_list = ee.List([ee.Reducer.percentile([25]),
                        ee.Reducer.percentile([75]),
                        ee.Reducer.min(),
                        ee.Reducer.max(),
                        ee.Reducer.mean()])

reducer_str_list = ee.List(['pr_p25', 'pr_p75', 'pr_min', 'pr_max', 'pr_mean'])
reducer_order = ee.List.sequence(0, 4)

start_year = 1985
end_year = 2070
global_years_list = ee.List.sequence(start_year, end_year)

modelfilter = ee.Filter.Or(
              ee.Filter.eq('scenario', 'historical'),
              ee.Filter.eq('scenario', 'rcp45'))

ic = ic.filter(modelfilter)

dict_list = []

for year in global_years_list.getInfo():

  print(year)
  start_year = year
  end_year = year
  years_list = ee.List.sequence(start_year, end_year)

  def year_fn(year):

    start = ee.Date.fromYMD(ee.Number(year), 1, 1)
    end = ee.Date.fromYMD(ee.Number(year).add(30), 1, 1)
    year_ic = ic.filterDate(start, end)

    def model_fn(model):

      model_ic = year_ic.filter(ee.Filter.eq('model', model))                     \
                        .select('pr')

      def month_fn(month):
        mo_im = model_ic.filter(ee.Filter.calendarRange(month, month,'month'))    \
                      .sum().multiply(86400).divide(30)                           \
                      .multiply(ee.Number(ndays_months.get(ee.Number(month).subtract(1))))
        mo_im = mo_im.clip(study_area)
        return mo_im

      ann_im = ee.ImageCollection(order_months.map(month_fn)).sum()
      return ann_im

    ann_ic = ee.ImageCollection(model_list.map(model_fn))

    def reducer_fn(re_i):
      re_i = ee.Number(re_i)
      re_im = ann_ic.reduce(reducer_list.get(re_i))
      mean_dict = re_im.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=study_area.geometry(),
        scale=500,
        maxPixels=1e10)
      avg_stat = mean_dict.get(reducer_str_list.get(re_i))
      return avg_stat

    stat_list = reducer_order.map(reducer_fn)

    out_ft = ee.Feature(None,
                        {'year':ee.Number(year),
                        'q25':stat_list.get(0),
                        'q75':stat_list.get(1),
                        'min':stat_list.get(2),
                        'max':stat_list.get(3),
                        'avg':stat_list.get(4)})

    ann_ic_list = ann_ic.toList(model_list.size())
    def reduce_fn(model_i):
      re_im = ee.Image(ann_ic_list.get(ee.Number(model_i)))
      mean_dict = re_im.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=study_area.geometry(),
        scale=500,
        maxPixels=1e10)
      avg_stat = mean_dict.get('pr')
      return avg_stat

    stat_list = model_order.map(reduce_fn)

    out_ft = out_ft.set(ee.Dictionary(model_list.zip(stat_list).flatten()))
    return out_ft

  ft_list = years_list.map(year_fn)

  ft = ee.Feature(ft_list.get(0))
  print(ft.getInfo())
  dict_list.append(ft.getInfo())

with open(out_file, 'w') as fo:

  model_list = model_list.getInfo()

  fo.write('yr,min,max,q25,q75,avg,' + ','.join(model_list) + '\n')

  for d in dict_list:
    yr = str(d['properties']['year'])
    min = str(d['properties']['min'])
    max = str(d['properties']['max'])
    q25 = str(d['properties']['q25'])
    q75 = str(d['properties']['q75'])
    avg = str(d['properties']['avg'])

    fo.write(','.join([yr, min, max, q25, q75, avg]))

    for model in model_list:
      fo.write(',' + str(d['properties'][model]))

    fo.write('\n')

1985


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 312.8597734034516, 'BNU-ESM': 321.5350589699216, 'CCSM4': 310.50888963051557, 'CESM1-BGC': 325.6754258915606, 'CNRM-CM5': 329.8889105035425, 'CSIRO-Mk3-6-0': 309.36064594177856, 'CanESM2': 316.12737824858823, 'GFDL-CM3': 321.04122814956554, 'GFDL-ESM2G': 332.61403911976714, 'GFDL-ESM2M': 304.82833519670936, 'IPSL-CM5A-LR': 319.40617415688075, 'IPSL-CM5A-MR': 328.9303579954857, 'MIROC-ESM': 336.29782016669583, 'MIROC-ESM-CHEM': 294.1476925903071, 'MIROC5': 318.84164840934216, 'MPI-ESM-LR': 315.2804522798884, 'MPI-ESM-MR': 325.526451590253, 'MRI-CGCM3': 321.16304275194796, 'NorESM1-M': 313.83835232119196, 'avg': 318.5993230412984, 'bcc-csm1-1': 323.3314793185987, 'inmcm4': 309.3826272311595, 'max': 343.75745974456845, 'min': 291.8085606371254, 'q25': 310.033291566378, 'q75': 326.1854965681699, 'year': 1985}}
1986


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 315.78097004639045, 'BNU-ESM': 321.02007851420797, 'CCSM4': 312.0421881477226, 'CESM1-BGC': 328.552648075365, 'CNRM-CM5': 328.74222409283243, 'CSIRO-Mk3-6-0': 311.3180583332451, 'CanESM2': 314.05512175921905, 'GFDL-CM3': 320.1223851732696, 'GFDL-ESM2G': 324.46673450351915, 'GFDL-ESM2M': 306.12262528273817, 'IPSL-CM5A-LR': 319.03522577150324, 'IPSL-CM5A-MR': 329.6208955771112, 'MIROC-ESM': 336.3684923630689, 'MIROC-ESM-CHEM': 294.0269268523408, 'MIROC5': 319.86133722479883, 'MPI-ESM-LR': 318.34920141760585, 'MPI-ESM-MR': 325.40389489600625, 'MRI-CGCM3': 320.6140102222502, 'NorESM1-M': 309.270775996488, 'avg': 318.32785726067766, 'bcc-csm1-1': 319.6278437334308, 'inmcm4': 310.48336449098156, 'max': 344.32906466810033, 'min': 292.5437238918522, 'q25': 310.1536222683799, 'q75': 325.7582603776121, 'year': 1986}}
1987


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 313.09104011862655, 'BNU-ESM': 317.08774701277724, 'CCSM4': 307.5213222336905, 'CESM1-BGC': 327.0495098422243, 'CNRM-CM5': 330.8461482344848, 'CSIRO-Mk3-6-0': 312.8140826456825, 'CanESM2': 313.46549185089344, 'GFDL-CM3': 316.53775789441306, 'GFDL-ESM2G': 318.68671420899557, 'GFDL-ESM2M': 308.3636152118449, 'IPSL-CM5A-LR': 320.0226783933415, 'IPSL-CM5A-MR': 333.7217592332873, 'MIROC-ESM': 339.2755887146055, 'MIROC-ESM-CHEM': 292.8814274261543, 'MIROC5': 317.0622477443983, 'MPI-ESM-LR': 320.92256810360135, 'MPI-ESM-MR': 326.0371492308616, 'MRI-CGCM3': 321.9540328902183, 'NorESM1-M': 309.5891651927289, 'avg': 318.05219043835564, 'bcc-csm1-1': 322.99344492857017, 'inmcm4': 309.1725080942639, 'max': 347.4887917112841, 'min': 291.3709988631301, 'q25': 309.4786223040586, 'q75': 326.1948240317966, 'year': 1987}}
1988


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 314.3807622804082, 'BNU-ESM': 316.72230391445447, 'CCSM4': 308.16706574143456, 'CESM1-BGC': 329.25305692884865, 'CNRM-CM5': 331.10846257997366, 'CSIRO-Mk3-6-0': 308.31654987631777, 'CanESM2': 313.5090311379599, 'GFDL-CM3': 314.311426641597, 'GFDL-ESM2G': 319.5444010203402, 'GFDL-ESM2M': 307.31684282488754, 'IPSL-CM5A-LR': 318.88844396602167, 'IPSL-CM5A-MR': 338.7159982680752, 'MIROC-ESM': 341.81758266129475, 'MIROC-ESM-CHEM': 289.7687372925911, 'MIROC5': 315.87301051966966, 'MPI-ESM-LR': 314.0624801635869, 'MPI-ESM-MR': 327.6274592176224, 'MRI-CGCM3': 321.82396234844754, 'NorESM1-M': 309.96379492254425, 'avg': 317.76085971636047, 'bcc-csm1-1': 321.80900853287943, 'inmcm4': 309.99767320463627, 'max': 351.5050934372031, 'min': 288.99246937812626, 'q25': 308.7611225496007, 'q75': 325.5638561963179, 'year': 1988}}
1989


...

{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 316.3521555586042, 'BNU-ESM': 325.9429972805552, 'CCSM4': 320.0434519056937, 'CESM1-BGC': 315.161754223614, 'CNRM-CM5': 355.1473335822534, 'CSIRO-Mk3-6-0': 329.95651981453415, 'CanESM2': 381.7639708548563, 'GFDL-CM3': 321.8949948410358, 'GFDL-ESM2G': 333.2314825363739, 'GFDL-ESM2M': 336.79188027209295, 'IPSL-CM5A-LR': 282.9674361720718, 'IPSL-CM5A-MR': 305.3112583303122, 'MIROC-ESM': 341.92332277940307, 'MIROC-ESM-CHEM': 333.0732572438014, 'MIROC5': 290.57452992873175, 'MPI-ESM-LR': 317.3288561442148, 'MPI-ESM-MR': 335.83258847505095, 'MRI-CGCM3': 353.2278497260209, 'NorESM1-M': 320.3026328518035, 'avg': 325.57556186606354, 'bcc-csm1-1': 299.32766182593144, 'inmcm4': 320.9308648403267, 'max': 390.8830649545392, 'min': 273.6877683792312, 'q25': 306.9442938296309, 'q75': 342.36738745117793, 'year': 2066}}
2067


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 313.52276885108824, 'BNU-ESM': 324.47738812823934, 'CCSM4': 322.40567715795095, 'CESM1-BGC': 318.43078926048054, 'CNRM-CM5': 353.75182260525065, 'CSIRO-Mk3-6-0': 332.6181608954665, 'CanESM2': 381.0867696465721, 'GFDL-CM3': 326.18517230075344, 'GFDL-ESM2G': 331.9020572695297, 'GFDL-ESM2M': 335.857684122894, 'IPSL-CM5A-LR': 282.17455466767467, 'IPSL-CM5A-MR': 306.9757937816001, 'MIROC-ESM': 347.3280990323553, 'MIROC-ESM-CHEM': 340.5492080739659, 'MIROC5': 293.7946606985865, 'MPI-ESM-LR': 316.98832905998614, 'MPI-ESM-MR': 333.24209635474375, 'MRI-CGCM3': 355.4443921179592, 'NorESM1-M': 321.85182369215215, 'avg': 326.85569142827376, 'bcc-csm1-1': 300.0842412883505, 'inmcm4': 325.2980309883498, 'max': 390.6341079082837, 'min': 274.86293671165106, 'q25': 307.7658549786328, 'q75': 344.19067560321406, 'year': 2067}}
2068


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 310.48834005495627, 'BNU-ESM': 320.5355723993673, 'CCSM4': 322.9568716672775, 'CESM1-BGC': 319.59253299926837, 'CNRM-CM5': 354.6137978041745, 'CSIRO-Mk3-6-0': 333.1867947263935, 'CanESM2': 378.3002396871896, 'GFDL-CM3': 320.1839938986539, 'GFDL-ESM2G': 329.86015415699893, 'GFDL-ESM2M': 332.90376270317483, 'IPSL-CM5A-LR': 288.56627983788695, 'IPSL-CM5A-MR': 307.3272768985711, 'MIROC-ESM': 347.3307924066857, 'MIROC-ESM-CHEM': 343.7477827752937, 'MIROC5': 293.32873870216923, 'MPI-ESM-LR': 320.0884447248526, 'MPI-ESM-MR': 335.84180459235296, 'MRI-CGCM3': 356.09384561306285, 'NorESM1-M': 330.09754341962895, 'avg': 327.4542669326238, 'bcc-csm1-1': 304.6153928900195, 'inmcm4': 326.8796436269744, 'max': 389.46131239511743, 'min': 276.90398298019295, 'q25': 308.85359782370665, 'q75': 344.13139976379733, 'year': 2068}}
2069


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 310.98642676960503, 'BNU-ESM': 325.0703981492224, 'CCSM4': 319.4733619812256, 'CESM1-BGC': 320.04542373073303, 'CNRM-CM5': 356.7700419339862, 'CSIRO-Mk3-6-0': 333.2277855746486, 'CanESM2': 378.15805876013656, 'GFDL-CM3': 315.8316012472815, 'GFDL-ESM2G': 332.6321939092266, 'GFDL-ESM2M': 335.9382388388484, 'IPSL-CM5A-LR': 291.4895429625995, 'IPSL-CM5A-MR': 307.4133575248842, 'MIROC-ESM': 350.4226330568419, 'MIROC-ESM-CHEM': 343.8329865156637, 'MIROC5': 294.36808006099176, 'MPI-ESM-LR': 317.668404049972, 'MPI-ESM-MR': 340.3210650355974, 'MRI-CGCM3': 359.8797907316463, 'NorESM1-M': 332.38373360312596, 'avg': 328.3149926248018, 'bcc-csm1-1': 305.9424144594193, 'inmcm4': 322.7593062254079, 'max': 391.9078622856813, 'min': 278.63134404607587, 'q25': 309.7460132003893, 'q75': 345.4854174370482, 'year': 2069}}
2070


{'type': 'Feature', 'geometry': None, 'properties': {'ACCESS1-0': 312.4853266211648, 'BNU-ESM': 326.03793980275344, 'CCSM4': 326.58424414661675, 'CESM1-BGC': 326.1731371061629, 'CNRM-CM5': 359.83044142658844, 'CSIRO-Mk3-6-0': 331.5187514955663, 'CanESM2': 375.33086478924946, 'GFDL-CM3': 313.25828264417305, 'GFDL-ESM2G': 334.70314315616093, 'GFDL-ESM2M': 336.3506113610336, 'IPSL-CM5A-LR': 296.71815302773797, 'IPSL-CM5A-MR': 310.8822081405296, 'MIROC-ESM': 346.99614921499955, 'MIROC-ESM-CHEM': 343.2280255383546, 'MIROC5': 293.5292083969291, 'MPI-ESM-LR': 313.7770683457713, 'MPI-ESM-MR': 339.9600079601063, 'MRI-CGCM3': 358.34464104283796, 'NorESM1-M': 336.74149048092, 'avg': 329.18436156167814, 'bcc-csm1-1': 307.8090865390652, 'inmcm4': 322.6128115585572, 'max': 388.8956900272305, 'min': 279.8286190203332, 'q25': 311.1568252265761, 'q75': 346.8011144948403, 'year': 2070}}
